In [6]:
import pong

import tensorflow as tf

In [56]:
SEQ_LEN = 88

next_batch = pong.parallel_batch_generator(
    bs=10,
    W=50,
    H=50,
    seq_len=SEQ_LEN,
    max_iter=100000,
    num_workers=1,
)

directions, controls, frames, game_over = next(next_batch)

directions.shape, controls.shape, frames.shape, game_over.shape

((10,), (10, 88, 2), (10, 88, 50, 50), (10, 88))

In [61]:
def mk_model():
    direction_inp = tf.keras.layers.Input(shape=(SEQ_LEN), name='dir_input')
    user_input = tf.keras.layers.Input(shape=(SEQ_LEN, 2), name='user_input')

    transformed_direction = tf.keras.layers.Dense(
        100,
        name='transformed_dir_input'
    )(direction_inp)
    
    transformed_user_input = tf.keras.layers.LSTM(
        100,
        return_sequences=True,
        name='transformed_user_inpuit'
    )(user_input)

    x = tf.keras.layers.LSTM(
        100,
        return_sequences=True,
        name='x'
    )(
        transformed_user_input,
        initial_state=transformed_direction
    )

    frames = tf.keras.layers.LSTM(2500, return_sequences=True)(x)
    game_over = tf.keras.layers.LSTM(1, return_sequences=True)(x)

    model = tf.keras.Model([direction_inp, user_input], [frames, game_over])
    
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )

    return model

In [62]:
mk_model()

ValueError: An `initial_state` was passed that is not compatible with `cell.state_size`. Received `state_spec`=ListWrapper([InputSpec(shape=(None, 100), ndim=2)]); however `cell.state_size` is [100, 100]

In [9]:
nn.LSTM()

torch.nn.modules.rnn.LSTM